# Titre du notebook

Intro ?

In [1]:
# Importation des packages
import numpy as np
import pandas
import pyspark
import math
import time

from operator import add

In [21]:
# Création de la matrice n lignes et p colonnes
mat= np.random.uniform(-1,1,size=(1000,100))
n=mat.shape[0]
p=mat.shape[1]
nb=range(n)
mat=np.insert(mat, 0, range(n), axis = 1)

In [22]:
mat

array([[  0.00000000e+00,  -7.65866808e-01,  -4.62671948e-01, ...,
         -3.47017100e-01,  -9.70742131e-01,   6.74716253e-01],
       [  1.00000000e+00,   4.81021624e-01,   7.38420888e-01, ...,
         -8.29526571e-01,  -1.72656581e-02,  -2.08563790e-01],
       [  2.00000000e+00,   5.30781146e-01,   4.30832564e-01, ...,
         -6.90015690e-01,  -5.32962990e-01,   9.10122190e-01],
       ..., 
       [  9.97000000e+02,   7.23705454e-01,   5.30645372e-01, ...,
         -6.41936621e-01,   8.59583362e-01,   5.91507491e-01],
       [  9.98000000e+02,  -4.46442610e-02,   1.23124409e-01, ...,
         -7.21262214e-01,  -5.42325811e-01,  -9.81334755e-01],
       [  9.99000000e+02,   6.90807523e-01,  -2.94574495e-01, ...,
         -5.11487969e-01,  -1.23024244e-01,   7.70728928e-01]])

In [23]:
np.savetxt("A.csv",mat,fmt='%1.3f',delimiter=',')
A=sc.textFile("A.csv").map(lambda line: line.split(",")).map(lambda line: [float(i) for i in line])

In [24]:
# Je l'ai mis en commentaire car doit-on paralléliser ?
#rdd=sc.parallelize(A,5) 
#rdd.take(5)
#new_rdd = rdd.flatMap(process_mat_row)
#new_rdd.take(12)

On modifie la matrice en un tableau de trois colonnes (index ligne, index colonne, valeur).

In [25]:
def process_mat_row(row):
    index = int(row[0])
    row = row[1:]
    return [[index, j, v] for j, v in enumerate(row)]

In [26]:
new_A = A.flatMap(process_mat_row)
new_A.take(12)

[[0, 0, -0.766],
 [0, 1, -0.463],
 [0, 2, 0.481],
 [0, 3, -0.372],
 [0, 4, -0.464],
 [0, 5, -0.341],
 [0, 6, -0.42],
 [0, 7, -0.012],
 [0, 8, -0.554],
 [0, 9, 0.233],
 [0, 10, -0.543],
 [0, 11, -0.657]]

## Algorithme 1

In [9]:
# On récupère tout d'abord les a_ij: on a ligne, (colonne,valeur): la clé est sur la ligne
def a_ij(row):
    return row[0], (row[1], row[2])

In [8]:
# On récupère ensuite les deux valeurs a_ij et a_ik associé au même i(clé) pour tous les j et k
mat_join_naive = new_A.map(a_ij).join(new_A.map(a_ij))
mat_join_naive.take(15)

[(0, ((0, 0.127), (0, 0.127))),
 (0, ((0, 0.127), (1, -0.349))),
 (0, ((0, 0.127), (2, -0.481))),
 (0, ((0, 0.127), (3, -0.828))),
 (0, ((0, 0.127), (4, 0.592))),
 (0, ((0, 0.127), (5, 0.85))),
 (0, ((0, 0.127), (6, 0.68))),
 (0, ((0, 0.127), (7, 0.038))),
 (0, ((0, 0.127), (8, -0.424))),
 (0, ((0, 0.127), (9, 0.763))),
 (0, ((1, -0.349), (0, 0.127))),
 (0, ((1, -0.349), (1, -0.349))),
 (0, ((1, -0.349), (2, -0.481))),
 (0, ((1, -0.349), (3, -0.828))),
 (0, ((1, -0.349), (4, 0.592)))]

In [10]:
# On fait la multiplication a_ij*a_ik, on garde les j et k en clé
def produit_matrice(row):
    index, ((j, vij), (k, vik)) = row
    return j, k, vij*vik

In [9]:
mat_produit_naive = mat_join_naive.map(produit_matrice)
mat_produit_naive.take(12)

[(0, 0, 0.016129),
 (0, 1, -0.044322999999999994),
 (0, 2, -0.061086999999999995),
 (0, 3, -0.105156),
 (0, 4, 0.075184),
 (0, 5, 0.10795),
 (0, 6, 0.08636),
 (0, 7, 0.0048259999999999996),
 (0, 8, -0.053848),
 (0, 9, 0.096901),
 (1, 0, -0.044322999999999994),
 (1, 1, 0.12180099999999998)]

## Algorithme 2

In [10]:
# On crée une nouvelle clé
mat_produit_key_naive = mat_produit_naive.map(lambda row: ((row[0], row[1]), row[2]))

In [11]:
# On additionne par clé
mat_finale_naive = mat_produit_key_naive.reduceByKey(add)

In [12]:
# Matrice finale 
AT_A_naive = mat_finale_naive.collect()

In [13]:
AT_A_naive.sort()
#AT_A #On obtient bien une matrice 10*10

In [14]:
liste_naive=[]
for k in range(p):
    liste_naive.append([AT_A_naive[k*p + i][1] for i in range(p)])

In [15]:
AT_A_finale_naive = np.array(liste_naive)
AT_A_finale_naive

array([[ 34.882255,   4.325688,   5.25007 ,   3.241141,   4.120881,
         -0.939737,  -4.315984,   1.56156 ,   3.370467,  -1.671469],
       [  4.325688,  34.438771,   1.244347,  -1.394023,   0.868624,
         -3.061998,  -3.296918,  -1.016092,   0.144422,   1.626774],
       [  5.25007 ,   1.244347,  26.5547  ,   0.626877,   3.967043,
          6.118294,  -3.997058,  -1.26915 ,   2.181615,  -3.056188],
       [  3.241141,  -1.394023,   0.626877,  30.517552,   2.403329,
         -1.173572,   1.473162,   1.435021,   1.680121,  -8.762421],
       [  4.120881,   0.868624,   3.967043,   2.403329,  37.231294,
         -4.50548 ,  -2.558421,   4.25732 ,   1.327155,  -7.698799],
       [ -0.939737,  -3.061998,   6.118294,  -1.173572,  -4.50548 ,
         27.299931,   3.633682,  -5.091089,  -2.99085 ,   2.547756],
       [ -4.315984,  -3.296918,  -3.997058,   1.473162,  -2.558421,
          3.633682,  36.772368,   1.191619,   2.914768,  -0.544911],
       [  1.56156 ,  -1.016092,  -1.26915

In [27]:
time1=time.clock()

mat_finale_naive = new_A.map(a_ij).join(new_A.map(a_ij)).map(produit_matrice).map(lambda row: ((row[0], row[1]), row[2])).reduceByKey(add)
AT_A_naive = mat_finale_naive.collect()
AT_A_naive.sort()

liste_naive=[]
for k in range(p):
    liste_naive.append([AT_A_naive[k*p + i][1] for i in range(p)])
AT_A_finale_naive = np.array(liste_naive)
    
time2=time.clock()
print(time2-time1)


0.10372299999999957


## Algorithme 3

In [16]:
new_A.take(12)

[[0, 0, 0.127],
 [0, 1, -0.349],
 [0, 2, -0.481],
 [0, 3, -0.828],
 [0, 4, 0.592],
 [0, 5, 0.85],
 [0, 6, 0.68],
 [0, 7, 0.038],
 [0, 8, -0.424],
 [0, 9, 0.763],
 [1, 0, 0.411],
 [1, 1, -0.418]]

In [12]:
def carre(row):
    r, c, aij = row
    return c,aij**2

def racine(row):
    c, aij = row
    return c,aij**1/2

In [18]:
norme=new_A.map(carre).reduceByKey(add).map(racine)

In [19]:
norme.take(10)

[(0, 17.441127499999993),
 (8, 17.127869500000003),
 (2, 13.277349999999998),
 (4, 18.615647000000003),
 (6, 18.386184),
 (1, 17.2193855),
 (3, 15.258776000000001),
 (9, 16.7236615),
 (5, 13.6499655),
 (7, 14.951690000000003)]

In [20]:
norme_c=norme.collect()
norme_c.sort()
n_c_df=sqlContext.createDataFrame(norme)
n_c=n_c_df.toPandas()

In [21]:
n_c.sort_values(by='_1',inplace=True)
n_c.reset_index(inplace=True,drop=True)
n_c

,_1,_2
0,0,17.441127
1,1,17.219386
2,2,13.277350
3,3,15.258776
4,4,18.615647
5,5,13.649966
6,6,18.386184
7,7,14.951690
8,8,17.127870
9,9,16.723661


In [13]:
# On récupère tout d'abord les a_ij: on a ligne, (colonne,valeur): la clé est sur la ligne
def a_ij(row):
    return row[0], (row[1], row[2])

In [22]:
# On récupère ensuite les deux valeurs a_ij et a_ik associé au même i(clé) pour tous les j et k
mat_join = new_A.map(a_ij).join(new_A.map(a_ij))
#mat_join.take(15)

In [14]:
# On fait la multiplication a_ij*a_ik, on garde les j et k en clé
def produit_matrice2(row):
    index, ((j, vij), (k, vik)) = row
    
    gamma=2*math.log(p)
    norme=1/(n_c['_2'][j]*n_c['_2'][k])
    proba=np.random.binomial(1,min(1,norme*gamma))
    vjk = (vij*vik)*proba
    
    return j, k, vjk

In [24]:
mat_produit = mat_join.map(produit_matrice2)
mat_produit.take(10)

[(0, 0, 0.016129),
 (0, 1, -0.0),
 (0, 2, -0.0),
 (0, 3, -0.0),
 (0, 4, 0.0),
 (0, 5, 0.0),
 (0, 6, 0.0),
 (0, 7, 0.0),
 (0, 8, -0.0),
 (0, 9, 0.0)]

## Algorithme 4

In [47]:
# On crée une nouvelle clé
mat_produit_key = mat_produit.map(lambda row: ((row[0], row[1]), (row[1],row[2])))
mat_produit_key.take(20)

[((0, 0), (0, 0.016129)),
 ((0, 1), (1, -0.0)),
 ((0, 2), (2, -0.0)),
 ((0, 3), (3, -0.0)),
 ((0, 4), (4, 0.0)),
 ((0, 5), (5, 0.0)),
 ((0, 6), (6, 0.0)),
 ((0, 7), (7, 0.0)),
 ((0, 8), (8, -0.0)),
 ((0, 9), (9, 0.0)),
 ((1, 0), (0, -0.0)),
 ((1, 1), (1, 0.0)),
 ((1, 2), (2, 0.0)),
 ((1, 3), (3, 0.0)),
 ((1, 4), (4, -0.0)),
 ((1, 5), (5, -0.29664999999999997)),
 ((1, 6), (6, -0.0)),
 ((1, 7), (7, -0.0)),
 ((1, 8), (8, 0.0)),
 ((1, 9), (9, -0.0))]

In [48]:
# On crée une nouvelle clé et on rajoute la valeur ci
#mat_produit_key2 = mat_produit.map(lambda row: ((row[0], row[1]), n_c['_2'][row[1]], row[2]))
#mat_produit_key2.take(20)

In [15]:
# On somme les vi
def somme_ligne(rowi,rowj):
    gamma=2*math.log(p)
    norme=1/(n_c['_2'][rowi[0]]*n_c['_2'][rowj[0]])
    bij = (rowi[1]+rowj[1])*(min(1,norme*gamma)==1)/norme+(rowi[1]+rowj[1])*(min(1,norme*gamma)!=1)/gamma
    return (rowi[0],bij)

In [69]:
# On additionne par clé
mat_finale = mat_produit_key.reduceByKey(somme_ligne).map(lambda row: ((row[0], row[1]), row[3]))

In [70]:
# Matrice finale 
AT_A = mat_finale.collect()

In [71]:
AT_A.sort()
AT_A #On obtient bien une matrice 10*10

[((0, 0), (0, 4.8560644377384227e-09)),
 ((0, 1), (1, 0.0)),
 ((0, 2), (2, -6.1910708295656424e-23)),
 ((0, 3), (3, 0.0)),
 ((0, 4), (4, 0.0)),
 ((0, 5), (5, -3.1988569483025608e-08)),
 ((0, 6), (6, 1.1940940530316107e-09)),
 ((0, 7), (7, 2.7034886184465532e-08)),
 ((0, 8), (8, -8.4034476664477441e-09)),
 ((0, 9), (9, -3.637698630745194e-08)),
 ((1, 0), (0, -1.4302704155430338e-05)),
 ((1, 1), (1, 5.6623099276981758e-23)),
 ((1, 2), (2, 0.0)),
 ((1, 3), (3, 8.3735234256666e-07)),
 ((1, 4), (4, 1.8862001849436907e-20)),
 ((1, 5), (5, -5.711322215251559e-12)),
 ((1, 6), (6, 0.0)),
 ((1, 7), (7, -0.0032555681609653729)),
 ((1, 8), (8, 3.6586071524467027e-24)),
 ((1, 9), (9, 6.2042208335499811e-11)),
 ((2, 0), (0, -1.0390079326839254e-11)),
 ((2, 1), (1, -3.4877456787050251e-08)),
 ((2, 2), (2, 1.6426924581529041e-12)),
 ((2, 3), (3, 1.6566551174714061e-11)),
 ((2, 4), (4, 0.0)),
 ((2, 5), (5, -1.2287336747526714e-07)),
 ((2, 6), (6, 2.1905905666094889e-05)),
 ((2, 7), (7, -3.1216569293127

In [72]:
liste=[]
for k in range(p):
    liste.append([AT_A[k*p + i][1] for i in range(p)])

In [73]:
AT_A_finale = np.array(liste)
AT_A_finale

array([[[  0.00000000e+00,   4.85606444e-09],
        [  1.00000000e+00,   0.00000000e+00],
        [  2.00000000e+00,  -6.19107083e-23],
        [  3.00000000e+00,   0.00000000e+00],
        [  4.00000000e+00,   0.00000000e+00],
        [  5.00000000e+00,  -3.19885695e-08],
        [  6.00000000e+00,   1.19409405e-09],
        [  7.00000000e+00,   2.70348862e-08],
        [  8.00000000e+00,  -8.40344767e-09],
        [  9.00000000e+00,  -3.63769863e-08]],

       [[  0.00000000e+00,  -1.43027042e-05],
        [  1.00000000e+00,   5.66230993e-23],
        [  2.00000000e+00,   0.00000000e+00],
        [  3.00000000e+00,   8.37352343e-07],
        [  4.00000000e+00,   1.88620018e-20],
        [  5.00000000e+00,  -5.71132222e-12],
        [  6.00000000e+00,   0.00000000e+00],
        [  7.00000000e+00,  -3.25556816e-03],
        [  8.00000000e+00,   3.65860715e-24],
        [  9.00000000e+00,   6.20422083e-11]],

       [[  0.00000000e+00,  -1.03900793e-11],
        [  1.00000000e+00,  -3

In [28]:
time1=time.clock()

norme=new_A.map(carre).reduceByKey(add).map(racine)
norme_c=norme.collect()
norme_c.sort()
n_c_df=sqlContext.createDataFrame(norme)
n_c=n_c_df.toPandas()
n_c.sort_values(by='_1',inplace=True)
n_c.reset_index(inplace=True,drop=True)

mat_finale = new_A.map(a_ij).join(new_A.map(a_ij)).map(produit_matrice2).map(lambda row: ((row[0], row[1]), (row[1],row[2]))).reduceByKey(somme_ligne).map(lambda row: ((row[0][0], row[0][1]), row[1][1]))


AT_A = mat_finale.collect()
AT_A.sort()
liste=[]
for k in range(p):
    liste.append([AT_A[k*p + i][1] for i in range(p)])
AT_A_finale = np.array(liste)

time2=time.clock()
print(time2-time1)

0.25105299999999975
